# Summarization - Conversation Summarization

The service provides summarization solutions for three types of genre: plain texts, conversations, and native documents. 

* **Text summarization** only accepts plain text blocks. 
* **Conversation summarization** accepts conversational input, including various speech audio signals. 
* **Native document summarization** accepts documents in their native formats, such as Word, PDF, or plain text

**Key features for text summarization**
Conversation summarization supports the following features:

* **Recap**: Summarizes a conversation into a brief paragraph.
* **Issue/resolution summarization**: Call center specific features that give a summary of issues and resolutions in conversations between customer-service agents and your customers.
* **Chapter title summarization**: Segments a conversation into chapters based on the topics discussed in the conversation, and gives suggested chapter titles of the input conversation.
* **Narrative summarization**: Generates detail call notes, meeting notes or chat summaries of the input conversation.

## Install Library

In [42]:
%pip install azure-ai-language-conversations==1.1.0

Note: you may need to restart the kernel to use updated packages.


## Load Azure Configurations

In [43]:
import os

# Load Azure configurations from environment variables
# Ensure that AZURE_AI_LANGUAGE_KEY and AZURE_AI_LANGUAGE_ENDPOINT are set in your environment
language_key = os.environ.get('AZURE_AI_LANGUAGE_KEY')
language_endpoint = os.environ.get('AZURE_AI_LANGUAGE_ENDPOINT')

## Create a Conversation Analysis client

In [44]:
from azure.ai.language.conversations import ConversationAnalysisClient
from azure.core.credentials import AzureKeyCredential

# Authenticate the client using Azure Key and Endpoint
def authenticate_client():
    """
    Authenticates the Azure Conversation Analytics client using the provided key and endpoint.

    Returns:
        ConversationAnalyticsClient: An authenticated client for Azure Conversation Analytics.
    """
    ca_credential = AzureKeyCredential(language_key)
    convesation_analysis_client = ConversationAnalysisClient(
        endpoint=language_endpoint,
        credential=ca_credential
    )
    return convesation_analysis_client

# Initialize the client
client = authenticate_client()

## Begin Conversation Analysis function

In [48]:
def conversation_analysis(client, conversations):
    """
    Analyzes multiple conversations using Azure's Conversation Analysis service.

    Args:
        client (ConversationAnalysisClient): The authenticated Azure Conversation Analysis client.
        conversations (list): A list of conversation objects to be analyzed.

    Returns:
        None: Prints the analysis results for each conversation.
    """
    # Start the conversation analysis task
    poller = client.begin_conversation_analysis(
        task={
            "displayName": "Analyze conversations from xxx",  # Task display name
            "analysisInput": {
                "conversations": conversations  # Input conversations to analyze
            },
            "tasks": [
                {
                    "taskName": "Summarize Conversation task",  # Task name
                    "kind": "ConversationalSummarizationTask",  # Task type
                    "parameters": {
                        "summaryAspects": [
                            "chapterTitle", "narrative", "issue", "resolution", "recap"
                        ]  # Aspects to summarize
                    },
                },
            ],
        }
    )

    # Wait for the task to complete and retrieve the result
    result = poller.result()
    task_results = result["tasks"]["items"]

    # Iterate through each task result
    for task in task_results:
        print(f"\n{task['taskName']} status: {task['status']}")  # Print task status
        task_result = task["results"]

        # Check for errors in the task result
        if task_result["errors"]:
            print("... errors occurred ...")
            for error in task_result["errors"]:
                print(error)
        else:
            # Process each conversation result
            for conversation_result in task_result["conversations"]:
                # Check for warnings in the conversation result
                if conversation_result["warnings"]:
                    print("... view warnings ...")
                    for warning in conversation_result["warnings"]:
                        print(warning)
                else:
                    # Print summaries for the conversation
                    print(f"\nConversation ID: {conversation_result['id']}")
                    summaries = conversation_result["summaries"]
                    for summary in summaries:
                        print(f"{summary['aspect']}: {summary['text']}")

In [46]:
conversations = [
    {
        "conversationItems": [
            {
                "text": "Hello, you’re chatting with Rene. How may I help you?",
                "id": "1",
                "role": "Agent",
                "participantId": "Agent_1"
            },
            {
                "text": "Hi, I tried to set up wifi connection for Smart Brew 300 coffee machine, but it didn’t work.",
                "id": "2",
                "role": "Customer",
                "participantId": "Customer_1"
            },
            {
                "text": "I’m sorry to hear that. Let’s see what we can do to fix this issue. Could you please try the following steps for me? First, could you push the wifi connection button, hold for 3 seconds, then let me know if the power light is slowly blinking on and off every second?",
                "id": "3",
                "role": "Agent",
                "participantId": "Agent_1"
            },
            {
                "text": "Yes, I pushed the wifi connection button, and now the power light is slowly blinking.",
                "id": "4",
                "role": "Customer",
                "participantId": "Customer_1"
            },
            {
                "text": "Great. Thank you! Now, please check in your Contoso Coffee app. Does it prompt to ask you to connect with the machine?",
                "id": "5",
                "role": "Agent",
                "participantId": "Agent_1"
            },
            {
                "text": "No. Nothing happened.",
                "id": "6",
                "role": "Customer",
                "participantId": "Customer_1"
            },
            {
                "text": "I’m very sorry to hear that. Let me see if there’s another way to fix the issue. Please hold on for a minute.",
                "id": "7",
                "role": "Agent",
                "participantId": "Agent_1"
            }
        ],
        "modality": "text",
        "id": "conversation1",
        "language": "en"
    },
    {
        "conversationItems": [
            {
                "text": "Hello, you’re chatting with Alex. How may I assist you today?",
                "id": "1",
                "role": "Agent",
                "participantId": "Agent_2"
            },
            {
                "text": "Hi, I'm having trouble connecting my Smart Home 500 thermostat to my wifi network.",
                "id": "2",
                "role": "Customer",
                "participantId": "Customer_2"
            },
            {
                "text": "I’m sorry to hear that. Let’s try to resolve this issue together. Could you please go to the settings menu on your thermostat and select 'Network Settings'? Then choose 'Wifi Setup' and let me know if you see a list of available networks.",
                "id": "3",
                "role": "Agent",
                "participantId": "Agent_2"
            },
            {
                "text": "Yes, I see a list of available networks.",
                "id": "4",
                "role": "Customer",
                "participantId": "Customer_2"
            },
            {
                "text": "Great! Please select your home wifi network from the list and enter your wifi password. Let me know if it connects successfully.",
                "id": "5",
                "role": "Agent",
                "participantId": "Agent_2"
            },
            {
                "text": "I’ve selected my network and entered the password, but it says 'Connection Failed'.",
                "id": "6",
                "role": "Customer",
                "participantId": "Customer_2"
            },
            {
                "text": "I apologize for the inconvenience. Let’s try another step. Please ensure that your wifi network is using a 2.4 GHz band, as the Smart Home 500 thermostat may not support 5 GHz networks. You can check this in your router settings. Could you verify this and try connecting again?",
                "id": "7",
                "role": "Agent",
                "participantId": "Agent_2"
            },
            {
                "text": "Okay, I checked and my network was on 5 GHz. I switched it to 2.4 GHz and tried connecting again. It worked! Thank you so much for your help!",
                "id": "8",
                "role": "Customer",
                "participantId": "Customer_2"
            },
            {
                "text": "You’re welcome! If you need any further assistance, feel free to reach out. Have a great day!",
                "id": "9",
                "role": "Agent",
                "participantId": "Agent_2"
            }
        ],
        "modality": "text",
        "id": "conversation2",
        "language": "en"
}

]

In [49]:
conversation_analysis(client, conversations)


Summarize Conversation task status: succeeded

Conversation ID: conversation1
issue: Customer is unable to set up wifi connection for Smart Brew 300 coffee machine.
resolution: Agent instructed customer to press wifi connection button, hold for 3 seconds, and check power light and Contoso Coffee app. No response from app. Agent checked for other solutions.
recap: The customer contacted the agent for assistance with setting up a wifi connection for their Smart Brew 300 coffee machine. The agent guided the customer through some troubleshooting steps, including holding the wifi connection button for 3 seconds and checking the power light. Despite these steps, the customer did not receive any prompts to connect with the machine via the Contoso Coffee app. The agent then decided to explore other potential solutions.
chapterTitle: Troubleshooting the Wi-Fi Connection for Smart Craft 300 Coffee Machine
narrative: Agent_1 and Customer_1 discussed an issue with setting up a wifi connection for